## Automação de Indicadores

**Objetivo**:
Desenvolver e implementar um projeto completo que envolva a automação de um processo realizado no computador.

**Descrição**:
Imagine que você trabalha em uma grande rede de lojas de roupas, com 25 lojas espalhadas por todo o Brasil.

Todos os dias, pela manhã, a equipe de análise de dados calcula os chamados "One Pages" e envia para o gerente de cada loja o "One Page" referente à sua loja, bem como todas as informações utilizadas no cálculo dos indicadores.

Um "One Page" é um resumo muito simples e direto, utilizado pela equipe de gerência da loja para conhecer os principais indicadores de cada loja. Ele permite, em uma única página (daí o nome "One Page"), tanto a comparação entre diferentes lojas quanto a análise dos indicadores que aquela loja conseguiu cumprir naquele dia ou não.

**Exemplo de "One Page"**:

Arquivos e Informações Importantes
Arquivo "Emails.xlsx" com o nome, a loja e o e-mail de cada gerente.

Obs: Sugiro substituir a coluna de e-mail de cada gerente por um e-mail seu, para poder testar o resultado.

Arquivo "Vendas.xlsx" com as vendas de todas as lojas. Obs: Cada gerente deve receber apenas o "One Page" e um arquivo anexo em Excel com as vendas de sua respectiva loja. As informações de outras lojas não devem ser enviadas ao gerente que não pertence àquela loja.

Arquivo "Lojas.csv" com o nome de cada loja.

Ao final, sua rotina deve enviar um e-mail para a diretoria (as informações estão também no arquivo "Emails.xlsx") com dois rankings das lojas em anexo: um ranking diário e outro anual. Além disso, no corpo do e-mail, deve destacar qual foi a melhor e a pior loja do dia, bem como a melhor e a pior loja do ano. O ranking de uma loja é determinado pelo seu faturamento.

As planilhas de cada loja devem ser salvas dentro da pasta correspondente à loja, com a data da planilha, a fim de criar um histórico de backup.

**Indicadores do "One Page":**
- Faturamento -> Meta Anual: 1.650.000 / Meta Diária: 1000
- Diversidade de Produtos (quantos produtos diferentes foram vendidos naquele período) -> Meta Anual: 120 / Meta Diária: 4
- Ticket Médio por Venda -> Meta Anual: 500 / Meta Diária: 500

**` Cada indicador deve ser calculado tanto diariamente quanto anualmente. O indicador diário deve referir-se ao último dia disponível na planilha de vendas (a data mais recente). `**

Bibliotecas

In [ ]:
import pandas as pd
import pathlib
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
import os

Base de Dados

In [ ]:
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', sep=';', encoding='latin1')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

display(emails)
display(lojas)
display(vendas)

Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [ ]:
vendas = vendas.merge(lojas, on='ID Loja')
display(vendas)

In [ ]:
dicionario_lojas = {}

for loja in lojas['Loja']:
  dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]
  
display(dicionario_lojas['Iguatemi Esplanada'])
display(dicionario_lojas['Shopping Vila Velha'])

Definindo o Dia do Indicador

In [ ]:
dia_indicador = vendas['Data'].max()
print(dia_indicador)
print(f'{dia_indicador.day}/{dia_indicador.month}')

Salvar a planilha na pasta de Backup

In [ ]:
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo

    dicionario_lojas[loja].to_excel(local_arquivo)

Meta

In [ ]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

Enviar e-mail para todas as Lojas

In [ ]:
for loja in dicionario_lojas:

    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

    #faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    

    #diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    #ticket medio
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()

    #ticket_medio_dia
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()


    # Configurações do email
    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    fromaddr = "seuemailaqui@gmail.com" # seu email
    toaddr = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    body = f'''
      <p>Bom dia, {nome}</p>

      <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong>, da <strong>Loja {loja}</strong> foi:</p>

      <table>
        <tr>
          <th>Indicador</th>
          <th>Valor Dia</th>
          <th>Meta Dia</th>
          <th>Cenário Dia</th>
        </tr>
        <tr>
          <td>Faturamento</td>
          <td style="text-align: center">R$ {faturamento_dia:.2f}</td>
          <td style="text-align: center">R$ {meta_faturamento_dia:.2f}</td>
          <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
        </tr>
        <tr>
          <td>Diversidade de Produtos</td>
          <td style="text-align: center">{qtde_produtos_dia}</td>
          <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
          <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
        </tr>
        <tr>
          <td>Ticket Médio</td>
          <td style="text-align: center">R$ {ticket_medio_dia:.2f}</td>
          <td style="text-align: center">R$ {meta_ticketmedio_dia:.2f}</td>
          <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
        </tr>
      </table>
        <br>
      <table>
        <tr>
          <th>Indicador</th>
          <th>Valor Ano</th>
          <th>Meta Ano</th>
          <th>Cenário Ano</th>
        </tr>
        <tr>
          <td>Faturamento</td>
          <td style="text-align: center">R$ {faturamento_ano:.2f}</td>
          <td style="text-align: center">R$ {meta_faturamento_ano:.2f}</td>
          <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
        </tr>
        <tr>
          <td>Diversidade de Produtos</td>
          <td style="text-align: center">{qtde_produtos_ano}</td>
          <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
          <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
        </tr>
        <tr>
          <td>Ticket Médio</td>
          <td style="text-align: center">R$ {ticket_medio_ano:.2f}</td>
          <td style="text-align: center">R$ {meta_ticketmedio_ano:.2f}</td>
          <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
        </tr>
      </table>

      <p>Seque em anexo a planilha com todos os dados para mais detalhes.</p>

      <p>Qualquer dúvida estou à disposição</p>
      <p>Att.; Gabriel</p>
    '''

    # Configurações do anexo
    filename = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    filepath = os.path.join(pathlib.Path.cwd(), caminho_backup, loja, filename)

    # Configuração da mensagem
    msg = MIMEMultipart()
    msg['From'] = fromaddr
    msg['To'] = toaddr
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'html'))

    # Configuração do anexo
    attachment = open(filepath, "rb")
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

    msg.attach(part)

    # Enviando o email
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(fromaddr, "suasenhadeautenticacao") # senha de autenticação
    text = msg.as_string()
    server.sendmail(fromaddr, toaddr, text)
    server.quit()

    print('E-mail da Loja {} enviado'.format(loja))

Ranking para Diretoria

In [ ]:
faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo), index=False)

vendas_dia = vendas.loc[vendas['Data']==dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_dia)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo), index=False)

Enviar e-mail para Diretoria

In [ ]:
# Configurações do email
fromaddr = "seuemailaqui@gmail.com" # seu email
toaddr = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'

body = f'''
<p>Prezados, bom dia!</p>

<p>Melhor loja do Dia: Loja {faturamento_lojas_dia.index[0]} com faturamento de R$ {faturamento_lojas_dia.iloc[0, 0]:.2f}<p>
<p>Pior loja do Dia: Loja {faturamento_lojas_dia.index[-1]} com faturamento de R$ {faturamento_lojas_dia.iloc[-1, 0]:.2f}<p>

<p>Melhor loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[0]} com faturamento de R$ {faturamento_lojas_ano.iloc[0, 0]:.2f}<p>
<p>Pior loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[-1]} com faturamento de R$ {faturamento_lojas_ano.iloc[-1, 0]:.2f}<p>

<p>Segue anexo os rankings do ano e do dia de todas as lojas.</p>

<p>Qualquer dúvida estou à disposição.</p>

<p>Att.;</p>

<p>Gabriel</p>
'''

# Configurações do anexo
filenames = [f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx',
             f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx']

# Configuração da mensagem
msg = MIMEMultipart()
msg['From'] = fromaddr
msg['To'] = toaddr
msg['Subject'] = subject
msg.attach(MIMEText(body, 'html'))

for filename in filenames:
    filepath = os.path.join(pathlib.Path.cwd(), caminho_backup, filename)

    # Configuração do anexo
    attachment = open(filepath, "rb")
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

    msg.attach(part)

# Enviando o email
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(fromaddr, "suasenhadeautenticacao") # senha de autenticação
text = msg.as_string()
server.sendmail(fromaddr, toaddr, text)
server.quit()

print('E-mail da Diretoria enviado')